In [1]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames
using Plots

In [2]:
#load social and ecological data and orgnize 
FL = repeat([Float64], inner=15)
dtype = append!([String], FL);

regional_EF = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/new_EF_SC1.csv",header=1,delim=",", types=dtype) |> DataFrame    
capacity = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/capacity2.csv",header=1,delim=",", types=dtype) |> DataFrame    
distance = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/distance.csv",header=1,delim=",") |> DataFrame 
LCA_model = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/LCA_model2.csv",header=1,delim=",") |> DataFrame 
D_Dsoc = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/D_Dsoc1.csv",header=1,delim=",") |> DataFrame
GDP = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/GDP.csv",header=1,delim=",") |> DataFrame;
emi_sink = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/SC_regional/emission_sink1.csv",header=1,delim=",") |> DataFrame;

In [3]:
cell_demand = 0.001*164.98*(1.3*1e6)*2           # annual demand of Li battery for tesla (1.369M EV/yr, ~2 NMC111 pack/EV, 226 kg/pack (35kwh/pack), 80~100 kWh per EV)

global_sink = 1.099e10                        # global pub (ocean) CO2 sequestration (ton/yr)
global_sink_tot = 2.236e10                  # global total (ocean+land) CO2 sequestration (ton/yr)
global_emi = 3.53e10                          # global CO2 emission (ton/yr)
es_ratio = global_sink/global_emi
es_ratio_tot = global_sink_tot/global_emi
emission_c = emi_sink[!, "emission"]          # national CO2 emission (ton/yr)
sink_c = emi_sink[!, "sink ton/yr"]           # national CO2 sink (ton/yr)

D = D_Dsoc[!, "D"]          # national CO2 emission (ton/yr)
Dsoc = D_Dsoc[!, "Dsoc ton/yr"]           # national CO2 sink (ton/yr)

EF_trans = 1.005/10000                        # ton CO2/km*ton (The average freight truck in the U.S. emits 161.8 grams of CO2 per ton-mile)
process = LCA_model[!,"process"]
countries = capacity[!,"country"]
ncty = size(countries,1)                          # No. of countries
nproc = size(process,1);                          # No. of processes 

mkt_loc = findfirst(isequal("United States"), countries)
mkt_proc = findfirst(isequal("battery"), process)

# seperate model
cathode = 1:4
cell = 5:10
noncell = [12,13]
battery = [11,14]
scaler = LCA_model[!,"scaler"]
price = LCA_model[!,"price (usd/ton product)"]
vGDP = GDP[!,"GDP usd"];

In [4]:
up_cath = scaler[1:4] * scaler[5] * scaler[11]
up_cell = scaler[5:10] * scaler[11]
cell_sef = scaler[11]
up_noncell = scaler[12:13] * scaler[14]
noncell_sef = scaler[14]
battery_sef = scaler[15];

In [5]:
input_amount = []
input_amount = vcat(up_cath, up_cell, cell_sef, up_noncell, noncell_sef, battery_sef) .* cell_demand
input_amount

MktV = price .* input_amount ;  # ton

In [6]:
input_amount

15-element Vector{Float64}:
  47166.82181639999
  66008.92034879999
  64284.806757599996
  65885.769378
 123150.9708
  68728.1933
   4975.796799999999
   1243.9491999999998
  29543.793499999992
  57221.66319999999
 310987.29999999993
   2144.74
   1715.792
 428947.99999999994
 428947.99999999994

----

In [141]:
model = Model(Gurobi.Optimizer);

Academic license - for non-commercial use only - expires 2023-11-27


In [142]:
#variables 
@variable(model, x[1:ncty, 1:nproc] >= 0)            # x[i,k] production amount of product k at location i
@variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)   # y[i,j,k] ship product k from i to j 
@variable(model, delta[1:ncty], Bin);

In [143]:
# capacity constraint: x[i,k] <= capacity[i][k]
for k in 1:nproc
    for i in 1:ncty
        @constraint(model, x[i,k] <= capacity[!, 2:end][i,k])
    end
end


# node output flow constraint
for k in 1:nproc
    for i in 1:ncty
        @constraint(model, sum(y[i,j,k] for j in 1:ncty) == x[i,k])
    end
end


# node output flow to market y[i,38, 15]
for i in 1:ncty
    for j in [e for e in 1:ncty if e != mkt_loc]
        @constraint(model, y[i,j,mkt_proc] == 0)
    end
end


for k in 1:nproc-1
    @constraint(model, sum(x[i,k] for i in 1:ncty) == input_amount[k])
end


# final demand constraint
@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == cell_demand);

In [144]:
# cathode LCA constraints (index=5)
for k in cathode
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,5] * scaler[k])
    end
end


# cell LCA constraints (index=11)
for k in cell
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,11] * scaler[k])
    end
end


# non cell LCA constraints (index=14)
for k in noncell
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,14] * scaler[k])
    end
end


# battery LCA constraints (index=15)
for k in battery
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,15] * scaler[k])
    end
end

In [145]:
# 1. production emission (TES)
val_x = zeros(ncty, nproc)

for k in 1:nproc
    for i in 1:ncty
        val_x[i,k] = regional_EF[i, k+1] * (1 - sink_c[i]/emission_c[i] - es_ratio)
    end
end

proc_emi = sum(x .* val_x);

In [146]:
# 2. transpotration emission (TES, w/ supply)
trans_emi = 0

for i in 1:ncty
    for j in 1:ncty
        amount = sum(y[i,j,k] for k in 1:nproc)
        arc_emi = amount * distance[!, 2:end][i,j] * EF_trans
        arc_seq = arc_emi * (sink_c[j]/emission_c[j] + es_ratio)
        arc_net = arc_emi - arc_seq
        trans_emi += arc_net
    end
end

In [147]:
slack = 1e-10
Pemi = 0
Pincome = 0
proD = Vector{AffExpr}(undef, ncty)
proM = Vector{AffExpr}(undef, ncty)

for i in 1:ncty
    for k in 1:nproc
        Pemi += regional_EF[i,k+1] * x[i,k]
        Pincome += x[i,k] * price[k]
    end
    proD[i] = Pemi
    proM[i] = Pincome
end


Temi = 0
transD = Vector{AffExpr}(undef, ncty)
for i in 1:ncty
    for j in 1:ncty
        amount = sum(y[i,j,k] for k in 1:nproc)
        Temi += amount * distance[!, 2:end][i,j] * EF_trans
    end
    transD[i] = Temi
end


for i in 1:ncty
    @constraint(model, proM[i] <= vGDP[i])
end

M = 1e15
Allo_soc = proM ./ vGDP .* Dsoc
SJ = proD + transD - Allo_soc

for i in 1:ncty
    @constraint(model, SJ[i] >= (delta[i] - 1) * M)
    @constraint(model, SJ[i] + slack <= delta[i] * M)
end

In [148]:
@objective(model, Max, sum(delta));

In [149]:
JuMP.optimize!(model)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 2295 rows, 13980 columns and 454275 nonzeros
Model fingerprint: 0x05b2ed93
Variable types: 13950 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [2e-04, 1e+15]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-10, 1e+15]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 1993 rows and 12497 columns
Presolve time: 0.18s
Presolved: 302 rows, 1483 columns, 11372 nonzeros
Variable types: 1479 continuous, 4 integer (4 binary)

Root relaxation: objective 2.700000e+01, 74 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      27.0000000   27.000

In [137]:
solution_x = JuMP.value.(x)
solution_y = JuMP.value.(y)
solution_delta = JuMP.value.(delta);

In [138]:
Pemi = 0
Pincome = 0
proD = zeros(ncty)
proM = zeros(ncty)

for i in 1:ncty
    for k in 1:nproc
        Pemi += regional_EF[i,k+1] * solution_x[i,k]
        Pincome += solution_x[i,k] * price[k]
    end
    proD[i] = Pemi
    proM[i] = Pincome
end


Temi = 0
transD = zeros(ncty)
for i in 1:ncty
    for j in 1:ncty
        amount = sum(solution_y[i,j,k] for k in 1:nproc)
        Temi += amount * distance[!, 2:end][i,j] * EF_trans
    end
    transD[i] = Temi
end


In [139]:
Allo_soc = proM ./ vGDP .* Dsoc
SJ = proD + transD - Allo_soc

30-element Vector{Float64}:
       0.0
  146302.13332199663
  123831.42101741045
  206256.08675930544
  159912.63574448245
       1.056562300404097e6
  875207.1441314534
 -794987.1116928686
       2.924519861638376e6
       3.0915770757717355e6
       3.950255987924516e6
  615154.6924511166
       3.1179878368311003e6
       ⋮
       5.53059013545199e6
       4.554944546771766e6
       3.2063561278521023e6
       5.516343264027968e6
      -9.0869063574939e8
      -3.938191876841539e8
       4.066881103618127e6
      -5.091918499342205e8
       6.573037055059096e6
       5.237567647721427e6
       7.624298300202082e6
       1.0146016860700918e7

In [140]:
res = 0
for i in SJ
    if i < 0
        res += 1
    end
end
res

4

In [106]:
delta = [0,1];

In [107]:
test = []
for i in 1:ncty
    if SJ[i] >= (delta[1]-1)*M && SJ[i]<delta[1]*M
        push!(test, delta[1])
    else
        push!(test, delta[2])
    end
end

In [108]:
test

30-element Vector{Any}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 1
 1
 1
 1
 0
 0
 1
 0
 1
 1
 1
 1

In [109]:
sum(test)

27